# Homework 1: Information Retrieval
## Instructions
1. Students will form teams of three people each and submit a single homework for each team in the format - ID1_ID2_ID3.ipynb
2. Groups of three!
2. **Do not write your names anywhere.**
3. For the code part: 
> **Write your code only in the mentioned sections. Do not change the code of other sections**. Do not use any imports unless we say so.
4. For theoretical questions, if any - write your answer in the markdown cell dedicated to this task, in **English**.


#### Deviation from the aforementioned  instructions will lead to reduced grade
---


## Clarifications
1. The same score for the homework will be given to each member of the team.  
2. The goal of this homework is to test your understanding of the concepts presented in the lectures. \
Anyhow, we provide here detailed explanations for the code part and if you have problems - ask.
3. Questions can be sent to the forum, you are encouraged to ask questions but do so after you have been thinking about your question. 
4. The length of the empty gaps (where you are supposed to write your code) is a recommendation (the amount of space took us to write the solution) and writing longer code will not harm your grade. We do not expect you to use the programming tricks and hacks we used to make the code shorter.   
Having said that, we do encourage you to write good code and keep that in mind - **extreme** cases may be downgraded.  
We also encourage to use informative variable names - it is easier for us to check and for you to understand. 

Since it is the first time we provide this homework please notify us if there is a bug/something is unclear, typo's exc..

5. We use Python 3.7 for programming.
6. Make sure you have all the packages and functions used in the import section. Most of it is native to Anaconda Python distribution.

### Have fun !

# Imports

In [1]:
import numpy as np
import pandas as pd
from collections import Counter
%matplotlib inline
from tqdm import tqdm
from typing import List,Dict
from IPython.display import Image
from IPython.core.display import HTML 
import time
import os
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer
import nltk
from string import punctuation, ascii_lowercase
from nltk.corpus import stopwords

nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yumi4\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yumi4\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
tic = time.perf_counter() #Start timer

# Config

In [3]:
cwd=os.getcwd()
INPUT_FILE_PATH = os.path.join(cwd, "lyrics.csv")
BOW_PATH = os.path.join(cwd, "bow.csv")
N_ROWS = 100000
CHUNCK_SIZE = 5000
tqdm_n_iterations = N_ROWS//CHUNCK_SIZE +1
COLS = [0]

## 1.1 Bag of words /TfIdf model
### Implement the following methods:

* `preprocess_sentence`: 
    * Lower case the word
    * Ignores it if it's in the stopwords list
    * Removes characters which are not in the allowed symbols
    * Stems it and appends it to the output sentence
    * Discards words with length <= 1
    
    
* `update_counts_and_probabilities`: 

    * Update self.unigram count (the amount of time each word is in the text)
    * Update self.bigram count (two consecutive word occurances)
    * Update inverted index: a dictionary with words as keys and the values is a dictionary - {'DocID' : word_count}   
    
* `compute_word_document_frequency`:

   * For each word count the number of docs it appears in. For example , for the word 'apple' -
$$\sum_{i \in docs} I(apple \in doc_i), I := Indicator function$$


* `update_inverted_index_with_tf_idf_and_compute_document_norm`:

    * Update the inverted index (which currently hold word counts) with tf idf weighing. We will compute tf by dividing with the number of words in each document. 
    * As we want to calculate the document norm, incrementally update the document norm. pay attention that later we apply sqrt to it to finish the process.

#### The result of this code is a bag of words model that already counts for TF-IDF weighing

In [4]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
allowed_symbols = set(l for l in ascii_lowercase)

In [5]:
def preprocess_sentence(sentence : str) -> List[str]:
    '''
    this function preprocesses a sentence by tokenizing it, removing stop words, stemming and removing punctuation and non-english words

    Parameters
    ----------
    sentence : str
        a string representing a sentence
    
    Returns
    -------
    List[str]
        a list of words
    '''
    output_sentence = []
    for word in word_tokenize(sentence):
        ### YOUR CODE HERE
        word = word.lower() # lower case the word
        for char in word:
            if char not in allowed_symbols:
                word = word.replace(char, '') # remove punctuation, keep only allowed characters
        if word not in stop_words and len(word) > 1: # remove stop_words
            # stem the word
            word = stemmer.stem(word)
            output_sentence.append(word)
        ### END YOUR CODE
    return output_sentence
    

def get_data_chuncks() -> List[str]:
    for i ,chunck in enumerate(pd.read_csv(INPUT_FILE_PATH, usecols = COLS, chunksize = CHUNCK_SIZE, nrows = N_ROWS)):
        chunck = chunck.values.tolist()
        yield [chunck[i][0] for i in range(len(chunck))] 

In [6]:
class TfIdf:
    def __init__(self):
        #unigram_count is a dictionary that maps a word to the number of times it appears in the corpus
        self.unigram_count =  Counter()
        #bigram_count is a dictionary that maps a bigram to the number of times it appears in the corpus
        self.bigram_count = Counter()
        #document_term_frequency is a dictionary that maps a document id to the number of words in the document
        self.document_term_frequency = Counter()
        #word_document_frequency is a dictionary that maps a word to the number of documents it appears in
        self.word_document_frequency = {}
        #inverted_index is a dictionary that maps a word to a dictionary that maps a document id to the tf-idf weight of the word in the document
        self.inverted_index = {}
        #doc_norms is a dictionary that maps a document id to the norm of the document
        self.doc_norms = {}
        #n_docs is the number of documents in the corpus
        self.n_docs = -1
        #sentence_preprocesser is a function that preprocesses a sentence
        self.sentence_preprocesser = preprocess_sentence
        #bow_path is the path to the bag of words
        self.bow_path = BOW_PATH

    def update_counts_and_probabilities(self, sentence:List[str], document_id:int) -> None:
        '''
        this function updates the unigram and bigram counts and the inverted index given a sentence and a document id

        Parameters
        ----------
        sentence : List[str]
            a list of words
        document_id : int
            the id of the document
        
        Returns
        -------
        None
        '''
        sentence_len = len(sentence)
        self.document_term_frequency[document_id] = sentence_len
        for i,word in enumerate(sentence):
            ### YOUR CODE HERE
            self.unigram_count.update([word])
            if i > 0:
                self.bigram_count.update([(sentence[i-1], sentence[i])])
            self.inverted_index.setdefault(word,{})
            self.inverted_index[word].setdefault(document_id,0)
            self.inverted_index[word][document_id] = self.unigram_count[word]
            ### END YOUR CODE
        
        
    def fit(self) -> None:
        for chunck in tqdm(get_data_chuncks(), total = tqdm_n_iterations):
            for sentence in chunck: #sentence is a song (string)
                self.n_docs += 1 
                if not isinstance(sentence, str):
                    continue
                sentence = self.sentence_preprocesser(sentence)
                if sentence:
                    self.update_counts_and_probabilities(sentence,self.n_docs)
        self.save_bow() # bow is 'bag of words'
        self.compute_word_document_frequency()
        self.update_inverted_index_with_tf_idf_and_compute_document_norm()
             
    def compute_word_document_frequency(self):
        '''
        this function computes the word document frequency for each word in the inverted index. e.g. if a word appears in 10 documents, the word document frequency is 10.
        '''
        print('compute_word_document_frequency')
        for word in self.inverted_index.keys():
            ### YOUR CODE HERE
            self.word_document_frequency[word] = len(self.inverted_index[word])
            ### END YOUR CODE
            
    def update_inverted_index_with_tf_idf_and_compute_document_norm(self):
        '''
        this function updates the inverted index with tf-idf weighting and computes the document norm for each document
        '''
        print('update_inverted_index_with_tf_idf_and_compute_document_norm')
        ### YOUR CODE HERE
        for word in self.inverted_index.keys():
            for document_id in self.inverted_index[word].keys():
                # Calculate tf:
                tf = self.inverted_index[word][document_id] / self.document_term_frequency[document_id]
                # Calculate idf:
                idf = np.log10(self.n_docs / (self.document_term_frequency[document_id]))
                self.inverted_index[word][document_id] = tf * idf
                
                self.doc_norms.setdefault(document_id,0)
                self.doc_norms[document_id] += self.inverted_index[word][document_id] ** 2
        ### END YOUR CODE
        for doc in self.doc_norms.keys():
            self.doc_norms[doc] = np.sqrt(self.doc_norms[doc]) 
            
    def save_bow(self):
        pd.DataFrame([self.inverted_index]).T.to_csv(self.bow_path)

In [7]:
tf_idf = TfIdf()
tf_idf.fit()

 95%|█████████▌| 20/21 [07:45<00:23, 23.30s/it]


compute_word_document_frequency
update_inverted_index_with_tf_idf_and_compute_document_norm


## 1.11 Bag of words model:

1. What is the computational complexity of this model, as a function of the number of docs in the corpus?
2. How can we make this code better in terms running time (parallelization or other topics you find)?

### YOUR SOLUTION HERE

Signs:
D – The number of documents
; V – The number of unique words
; m – The average number of words in a document

a. Iterating over all documents - O(D)

a1. For each document: pre-processing costs O(m) [preprocessing is performed for an average of m words in each document, when pre-processing for one word costs O(1)].

a2. For each document: we update_counts_and_probabilities which iterates over m words in a document when each update costs O(1) --> overall the update costs O(m).

Therefore the total complexity of a section is O(mD).

b. Saving bag of words costs O(1).

c. Computing word frequency costs O(V) since requires going over V words (all unique words).

d. Updating inverted index costs O(VD) - we go over all unique words (V) for each document (D), with O(1) calculation for each.
 
All together, the complexity of the model is O(mD) + O(V) + O(VD) = O(VD), and that is because V>>m.
This complexity is as we learned in the class.

------------------------------------------------------------------------------------------------------------------------

2.There are several ways we can make this code better in terms of running time:
Parallelization: We can parallelize the code using multiprocessing or multithreading. For example,
We can split for x chunks, and work in parallel for every chunk, and then merge the results. For example, we can heuristicaly choose to pull [top K]/[num of chunks] from each chunk instead of [top K] from all together.

Optimizing the code: We can optimize the code by using more efficient data structures and algorithms. For example, we can use a hash table or a Trie data structure to store the inverted index instead of a dictionary. We can also use a sparse matrix representation for the term-document matrix to save memory and speed up computation. Finally, we can optimize the stemmer and tokenizer to make them faster and more efficient.





### END YOUR SOLUTION

## 1.2 DocumentRetriever
Not this retriever &#8595;


![dsafdsafsdafdsf](https://cdn3-www.dogtime.com/assets/uploads/2019/10/golden-cocker-retriever-mixed-dog-breed-pictures-cover-1.jpg)

### Implement the following methods:

`reduce_query_to_counts`: given a list of words returns a counter object with words as keys and counts as values.

`rank`: given a query and relevant documents calculate the similarity (cosine or inner product simialrity) between each document and the query.   
Make sure to transform the query word counts to tf idf as well. 

`sort_and_retrieve_k_best`: returns the top k documents.

In [8]:
class DocumentRetriever:
    def __init__(self, tf_idf):
        #sentence_preprocesser is a function that preprocesses a sentence
        self.sentence_preprocesser = preprocess_sentence  
        #vocab is a set of all the words in the corpus
        self.vocab = set(tf_idf.unigram_count.keys())
        #n_docs is the number of documents in the corpus
        self.n_docs = tf_idf.n_docs
        #inverted_index is a dictionary that maps a word to a dictionary that maps a document id to the tf-idf weight of the word in the document
        self.inverted_index = tf_idf.inverted_index
        #word_document_frequency is a dictionary that maps a word to the number of documents it appears in
        self.word_document_frequency = tf_idf.word_document_frequency
        #doc_norms is a dictionary that maps a document id to the norm of the document
        self.doc_norms = tf_idf.doc_norms
        
    def rank(self, query:Dict[str,int], documents:Dict[str,Counter], metric:str) -> Dict[str, float]:
        '''
        this function ranks the documents according to the query and the chosen similarity metric

        Parameters
        ----------
        query : Dict[str,int]
            a dictionary of words and their counts in the query
        documents : Dict[str,Counter]
            a dictionary of documents that contain the query words and their tf-idf weights
        metric : str
            the similarity metric to use. can be 'cosine' or 'dot'
        
        Returns
        -------
        Dict[str, float]
            a dictionary of documents and their similarity to the query
        '''
        result = {} # key: DocID , value : float , simmilarity to query
        query_len = np.sum(np.array(list(query.values())))
        ### YOUR CODE HERE
        query_norm = 0
        for qord in query:
            # for each word sum the counter of word **2
            query_norm += query[qord]**2
            sim=0
            for doc in documents[qord]:
                # caculate for each the inner product and add to the similarity ranking
                sim += query[qord]*documents[qord][doc]
                if doc not in result:
                    result[doc] = sim
                else:
                    result[doc] += sim
        query_norm = np.sqrt(query_norm)
        ### END YOUR CODE
        if metric == 'cosine':
            ### YOUR CODE HERE
             for doc in result.keys():
                result[doc] = result[doc]/(self.doc_norms[doc]*query_norm)
                
            ### END YOUR CODE
        return result
        
    
    def sort_and_retrieve_k_best(self, scores:Dict[str, float], k:int)-> List[str]:
        '''
        this function sorts the documents according to their similarity to the query and returns the k best documents
        '''
        ### YOUR CODE HERE
        return sorted(scores, key=scores.get, reverse=True)[:k]
        ### END YOUR CODE

    
    def reduce_query_to_counts(self, query:List)->  Counter:
        '''
        this function reduces the query to a dictionary of words and their counts
        '''
        ### YOUR CODE HERE
        return Counter(query)
        ### END YOUR CODE
        
        
    def get_top_k_documents(self, query:str, metric:str , k=5) -> List[str]:
        query = self.sentence_preprocesser(query)
        query = [word for word in query if word in self.vocab] # filter nan 
        query_bow = self.reduce_query_to_counts(query)
        relavant_documents = {word : self.inverted_index.get(word) for word in query}
        ducuments_with_similarity = self.rank(query_bow,relavant_documents, metric)
        return self.sort_and_retrieve_k_best(ducuments_with_similarity,k)
        
dr = DocumentRetriever(tf_idf)

In [9]:
from IPython.display import HTML
query = "Better stop dreaming of the quiet life, 'cause it's the one we'll never know And quit running for that runaway bus 'cause those rosy days are few And stop apologizing for the things you've never done 'Cause time is short and life is cruel but it's up to us to change This town called malice"
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/KT6ZtUbVw1M?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

C:\Users\yumi4\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\display.py:431: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [10]:
cosine_top_k = dr.get_top_k_documents(query, 'cosine')
print(cosine_top_k)
inner_product_top_k= dr.get_top_k_documents(query, 'inner_product')
print(inner_product_top_k)

[74518, 74537, 67610, 67748, 66206]
[99931, 99963, 99661, 99977, 99990]


In [11]:
for index, song in enumerate(pd.read_csv(INPUT_FILE_PATH,usecols = [0]).iloc[cosine_top_k]['Lyric']):
    sep = "#"*50
    print(F"{sep}\nsong #{index} \n{song} \n{sep}")

##################################################
song #0 
[INTRO]. My mama took me to Sam Goody's. I wanted to buy a 50 Cent CD. I took that shit home. That shit was wack like a muthafucka. Don't fuck with Game. I like 50 Cent. He reminds me Spongebob. And Tony Yayo is Blues Clues. And Lloyd Banks is Dora the Explorer. They're my friends. Psyche. I went down one of them Bodaga shits right there in Harlem. Got me a bootleg Lloyd Banks and Young Buck CD. Took that shit home, put it in my boom box. Thought I was bout to be on some radio Raheim shit. Man that shit sound like some Vanessa Williams '88. I mean Olivia cute but they say that bitch a man. So this Black Wallstreet for life now. GGG-UNOT!. [GAME]. 300 Bars and Runnin. Just loan me your ears for 15 minutes. Walk with me. Here the breakdown, pass the doja, .45 in the holster. Hollow tips'll fold 'em, them *****z they toy soldiers. Oh, that boy colder than Hova unless he sober. Like I'm the president, but this ain't the takeover. 

# 1.3 term statistics:
Use "tf_idf" object that we created earlier and answer the following questions:

1. How many unique words we have?
2. How many potential word bigrams we have? How many actual word bigrams we have? How do you explain this difference?
3. What is the storage size of the input file "lyrics.csv"? What is the output file (bow.csv) size? how do you explain this difference?  

In [12]:
# 1. 
### YOUR SOLUTION HERE
num_unique_words = len(tf_idf.unigram_count.keys())
print("The number of unique words we have in our chunk is " +str(num_unique_words))
### END YOUR SOLUTION

"""
### Your verbal solution here
The unigram method counts for each word how many times it appears in the chunk. Therefore, each key will be a unique word.
All together, the number of keys in the unigram dictionary will be the number of the unique words we have ion the chunk.
### End your verbal solution here
"""

# 2.
### YOUR SOLUTION HERE
actual_bigram_num = len(tf_idf.bigram_count.keys())
print("The number of bigrams we have in our chunk is " +str(actual_bigram_num))
### END YOUR SOLUTION

"""
### Your verbal solution here 
A bigram is a unique set of 2 words. Assuming we have n unique words, and that bigram can be a set of 2 occurances of the same word
(For examples, there are names of people/places which is the same word twice; like yah yah, bol bol, mohamed mohamed, etc).
For the first word in the set we have n options, and for the second word in the set we also have n options.
All together, the options sums up to be n^2.
We already know that we have 100331 words in the chunk, and therefore the potential bigrams we could have is 100331^2,
but as we can see, the actual number of bigrams is 2485009 (a much lower amount).
The reason for that is because in reality there is a certain logic for the order of the words, and some words will not be consecutive to the others,
therefor will not reach the potential 100331^2 number of combinations we can potentially have.
### End your verbal solution here
"""

# 3.
### YOUR SOLUTION HERE
import os
input_file_size = os.path.getsize("lyrics.csv")
print("Input file size: ", input_file_size, " bytes")
output_file_size = os.path.getsize("bow.csv")
print("Output file size: ", output_file_size, " bytes")
### END YOUR SOLUTION

"""
### Your verbal solution here
The storage size of "lyrics.csv" (will be refered to as lyrics) is much larger then the storage size of "bow.csv" (will be refered to as bow)
and that is because bow serves as a map between uniqe words, and the number of their occurances in each song, not saving the actuall full song lyrics.
Because of that, the number of characters we save is much smaller (reduced because of the pointers of the hash table).
In addition, some actions have been taken that reduces the number of words and the number of characters in each word we save in the lyrics file,
for example: stemming, stop-words removal, etc.
### End your verbal solution here
"""

The number of unique words we have in our chunk is 100331
The number of bigrams we have in our chunk is 2485009
Input file size:  168274338  bytes
Output file size:  94708742  bytes


'\n### Your verbal solution here\nThe storage size of "lyrics.csv" (will be refered to as lyrics) is much larger then the storage size of "bow.csv" (will be refered to as bow)\nand that is because bow serves as a map between uniqe words, and the number of their occurances in each song, not saving the actuall full song lyrics.\nBecause of that, the number of characters we save is much smaller (reduced because of the pointers of the hash table).\nIn addition, some actions have been taken that reduces the number of words and the number of characters in each word we save in the lyrics file,\nfor example: stemming, stop-words removal, etc.\n### End your verbal solution here\n'

## 1.4 NgramSpellingCorrector
Now we will implement a Ngarm (character Ngrams) spelling corrector. That is, we have an out of vocabulary word (v) and we want to retrieve the most similar words (in our vocabulary) to this word.
we will model the similarity of two words by-

$$JaccardIndex =  \frac{|X \cap Y|}{|X \cup Y|}$$

Where v is an out of vocabulary word (typo or spelling mistake), w is in a vocabulary word, X is the ngram set of v and Y is the ngram set of w.
For example, if n == 3, the set of ngrams for word "banana" is set("ban","ana","nan","ana") = {"ban","ana","nan"}

In order to do it efficently, we will first construct an index from the possible Ngrams we have seen in our corpus to the words that those Ngrams appear in, in order prevent comparing v to all of the words in our corpus.
Then, we will implement a function that computes this similarity.

* Make sure you compute the JaccardIndex efficently!

In [13]:
def get_bigrams(word):
    for ngram in nltk.ngrams(word, 2):
        yield "".join(list(ngram))
    
def get_bigrams_not_gen(word):
    return ["".join(list(ngram)) for ngram in nltk.ngrams(word, 2)]
"""
for example - get_bigrams is a generator, which is an object we can loop on:
for ngram in get_bigrams(word):
    DO SOMETHING
"""

'\nfor example - get_bigrams is a generator, which is an object we can loop on:\nfor ngram in get_bigrams(word):\n    DO SOMETHING\n'

In [14]:
class NgramSpellingCorrector:
    def __init__(self, unigram_counts: Counter, get_n_gram: callable):
        #unigram_counts is a dictionary that maps a word to its count in the corpus
        self.unigram_counts = unigram_counts
        #ngram_index is a dictionary that maps an ngram to a set of words that contain it
        self.ngram_index = {}
        #get_n_grams is a function that returns the ngrams of a word
        self.get_n_grams = get_n_gram
    
    def build_index(self) -> None:
        '''
        this function builds an index of ngrams to words that contain them
        the structure of the index is a dictionary of ngrams to sets of words
        '''
        ### YOUR CODE HERE
        for word in self.unigram_counts.keys():  
            for bigram in self.get_n_grams(word):
                if bigram not in self.ngram_index:
                    self.ngram_index[bigram] = set()
                self.ngram_index[bigram].add(word)
        ### END YOUR CODE
        
    def get_top_k_words(self,word:str,k=5) -> List[str]:
        '''
        this function returns the k most probable candidate words for a given out of vocabulary word

        Parameters
        ----------
        word : str
            the out of vocabulary word
        k : int, optional
            the number of candidates to return, by default 5
        
        Returns
        -------
        List[str]
            a list of the k most probable candidate words
        '''
        ### YOUR CODE HERE
        wrong_word_bigrams = get_bigrams_not_gen(word)
        replace_cand_words = {}
        for bi in wrong_word_bigrams:
            cand_words = self.ngram_index[bi]
            for cw in cand_words:
                if cw not in replace_cand_words.keys():
                    cand_word_bigrams = get_bigrams_not_gen(cw)
                    union_bi = len(set(wrong_word_bigrams).union(set(cand_word_bigrams)))
                    intersect_bi = len(set(wrong_word_bigrams).intersection(set(cand_word_bigrams)))
                    jack_sim = float(intersect_bi)/float(union_bi)
                    replace_cand_words[cw] = jack_sim
        sorted_cw = sorted(replace_cand_words, key=replace_cand_words.get,reverse=True)
        top_k_can_words = sorted_cw[:k]

        return top_k_can_words, replace_cand_words
    
        ### END YOUR CODE


class BigramSpellingCorrector(NgramSpellingCorrector):
    def __init__(self, unigram_counts: Counter):
        super().__init__(unigram_counts, get_bigrams)

Code Understanding Question:

In the "get top k words" function, does each candidate word have repeat ones or there is a channce that the same word will be iterated more than once? Explain your answer.

Answer here:

According to our algorithm, the incorrect word is decomposed into bigrams, and the candidate words for replacement are words with a certain bigram. Hence the same word can be candidated more than once, each time thanks to a different bigram.
For example, for the incorrect word "acress", the word "cress" can be candidated 4 time (thanks to 'cr' 're' 'es' 'ss').
But, calculating the jaccared score happens only once for each candidate word (we have a relevant condition in our algorithm). Therefore replace_cand_words will only include unique candidate words, and also the "top k words" list.

In [15]:
out_of_vocab_word = 'acress'
bigram_spelling_corrector = BigramSpellingCorrector(tf_idf.unigram_count)
bigram_spelling_corrector.build_index()
candidate_words, scores = bigram_spelling_corrector.get_top_k_words(out_of_vocab_word)

In [16]:
candidate_words

['cress', 'recress', 'ress', 'actress', 'cresson']

In [17]:
for word in candidate_words:
    print(f'The jaccared score for the word {word} is {round(scores[word],2)}')

The jaccared score for the word cress is 0.8
The jaccared score for the word recress is 0.67
The jaccared score for the word ress is 0.6
The jaccared score for the word actress is 0.57
The jaccared score for the word cresson is 0.57


# The End - You did it!

In [18]:
#The time it took to run the entire code
toc = time.perf_counter()
print(f"The time it took to run the entire code is: {(toc - tic)/60} minuts")

The time it took to run the entire code is: 8.125320970003182 minuts
